<a href="https://colab.research.google.com/github/Churchiill/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

# Import data

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/phot.tar.gz').extractall('/content/drive/MyDrive')

In [ ]:
import pandas as pd
from pathlib import Path
import csv

files = Path("/content/drive/MyDrive/phot/I").rglob("*.dat")

#f = open("/content/drive/MyDrive/shittt.csv", "x")
i = 0


for file in files:
  datContent = [i.strip().split() for i in open(file).readlines()]

  # write it as a new CSV file
  with open("/content/drive/MyDrive/shittt.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(datContent)

  df = pd.read_csv("/content/drive/MyDrive/shittt.csv", names=['a','b','c']) 
  if i==1:
    data = pd.concat([data,df])
  else:
    data = df
  
  i = 1



In [ ]:
data.reset_index(drop=True)

,a,b,c
0,2171.88226,18.316,0.035
1,2183.84165,18.256,0.045
2,2184.84118,17.774,0.026
3,2191.81204,17.933,0.019
4,2200.82188,18.295,0.027
...,...,...,...
39609,4441.66735,17.822,0.022
39610,4485.85627,17.260,0.016
39611,4492.84004,17.283,0.021
39612,4494.77086,17.852,0.021


In [18]:
aa = pd.concat([df1,df2], axis=1)
aa

,a,b,c,a,b,c
0,2123.93450,17.836,0.025,2187.85691,17.026,0.016
1,2130.92356,17.900,0.028,2194.82402,16.824,0.012
2,2131.92535,17.775,0.029,2206.83315,16.770,0.014
3,2132.92352,17.919,0.028,2216.82421,16.977,0.013
4,2134.92508,17.779,0.026,2225.75888,16.914,0.012
...,...,...,...,...,...,...
381,4530.63068,17.948,0.026,NaN,NaN,NaN
382,4532.65643,17.999,0.027,NaN,NaN,NaN
383,4536.62867,17.723,0.020,NaN,NaN,NaN
384,4539.51064,17.911,0.026,NaN,NaN,NaN


In [ ]:
import csv

# read .dat to a list of lists
datContent = [i.strip().split() for i in open("/content/drive/MyDrive/phot/I/OGLE-LMC-ACEP-074.dat").readlines()]

#f = open("/content/drive/MyDrive/shitt.csv", "x")

# write it as a new CSV file
with open("/content/drive/MyDrive/shitt.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(datContent)

df2 = pd.read_csv("/content/drive/MyDrive/shitt.csv", names=['a','b','c']) 
df2.tail()
